In [62]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src.dbc import TimeDivisionKMeans
from src.dbc.utils import data_preprocessing, dimension_reduction, calc_contribution

In [2]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
df = data_preprocessing(xlsx)

# 3. dimension_reduction
df = dimension_reduction(df)

# 4. day clustering
copy_df = df.copy()

In [3]:
# 4. day clustering
df = copy_df[:24]
size = 3
tdKMeans = TimeDivisionKMeans(df, size=size)

df.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2018-05-01 00:00:00,0.341,0.121,0.245,0.009,0.085,0.154,0.161,0.311,0.358,0.177,...,0.372,0.181,0.322,0.374,0.571,0.453,0.161,0.127,0.259,0.364
2018-05-01 01:00:00,0.337,0.186,0.285,0.010,0.071,0.127,0.179,0.225,0.206,0.178,...,0.245,0.238,0.292,0.382,0.327,0.352,0.172,0.133,0.234,0.286
2018-05-01 02:00:00,0.324,0.214,0.487,0.009,0.061,0.139,0.182,0.221,0.150,0.152,...,0.245,0.250,0.287,0.320,0.359,0.366,0.242,0.219,0.197,0.368
2018-05-01 03:00:00,0.319,0.250,0.300,0.009,0.067,0.132,0.172,0.274,0.142,0.205,...,0.177,0.223,0.329,0.170,0.322,0.267,0.269,0.164,0.229,0.414
2018-05-01 04:00:00,0.235,0.199,0.107,0.009,0.047,0.131,0.153,0.255,0.139,0.169,...,0.129,0.171,0.301,0.168,0.351,0.255,0.170,0.132,0.198,0.318


In [4]:
total_size = len(df)
division_df = [df[_:_ + size]
                            for _ in range(0, total_size, size)]

In [5]:
mean_pattern = division_df[0].mean(axis=1)

In [6]:
def tss(mean_pattern, df):
    A = np.expand_dims(mean_pattern.values, axis=0)
    tss = 0 
    for col in df:
        B = np.expand_dims(df[col].values, axis=0)
        tss += euc(
                B,
                A
            )[0][0] ** 2
    return tss

In [7]:
tss(mean_pattern, division_df[0])

8.379768472222231

In [ ]:
tdKMeans.tss_list

In [9]:
tdKMeans.init_setting()
hc, ci = tdKMeans.run()

hc.head()

8 / 8 ==> 83.821073115768


,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,4.0,2.0,3.0,6.0,6.0,6.0,2.0,2.0,2.0,2.0,...,4.0,2.0,4.0,4.0,0.0,0.0,2.0,2.0,2.0,4.0
1,6.0,4.0,4.0,5.0,5.0,2.0,2.0,6.0,2.0,2.0,...,4.0,4.0,0.0,4.0,0.0,6.0,4.0,4.0,4.0,0.0
2,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,...,3.0,2.0,6.0,4.0,1.0,4.0,4.0,0.0,6.0,4.0
3,5.0,5.0,6.0,6.0,5.0,3.0,3.0,3.0,1.0,5.0,...,5.0,1.0,3.0,0.0,4.0,0.0,5.0,3.0,3.0,0.0
4,6.0,1.0,1.0,1.0,1.0,6.0,5.0,6.0,0.0,6.0,...,5.0,3.0,3.0,0.0,4.0,5.0,1.0,6.0,5.0,2.0


In [ ]:
test = pd.DataFrame(df.values.reshape(-1, 3, 108).sum(axis=1),
                    columns=df.columns)
test.head()

In [ ]:
def calc_contribution(hc, ci, df):
    _hc = hc.copy()
    _test = list()

    for col in _hc:
        _hc_info = _hc[col].copy()
        _hc_pattern = df[col].copy()

        contributions = list()
        for division_round, _ in enumerate(_hc_info):
            conts = ci[division_round][1]
            cont = conts[int(_)]
            _c_pattern = ci[division_round][0][int(_)]
            _h_pattern = _hc_pattern[division_round]

            if _c_pattern.sum() <= _h_pattern.sum():
                upper_conts = conts[conts > cont]
                if len(upper_conts) == 0:
                    contributions.append(
                        cont
                    )
                else:
                    upper_cont = upper_conts[upper_conts.argsort()][0]
                    
                    _conts = np.arange(upper_cont, cont)
                    _conts_max = len(_conts)
                
                    _percentage = (_h_pattern.sum() - _c_pattern.sum()) / _c_pattern.sum()
                    _new_cont_idx = round(_conts_max * _percentage)

                    if _new_cont_idx == _conts_max:
                        contributions.append(
                            cont
                        )
                    else:
                        contributions.append(
                            _conts[_new_cont_idx]
                        )
            else:
                lower_conts = conts[conts < cont]
                if len(lower_conts) == 0:
                    contributions.append(
                        cont
                    )
                else:
                    lower_cont = lower_conts[lower_conts.argsort()][::-1][0]
                    _conts = np.arange(lower_cont, cont)
                    _conts_max = len(_conts)
                    _percentage = _h_pattern.sum() / _c_pattern.sum()

                    _new_cont_idx = round(_conts_max * _percentage)
                    if _new_cont_idx == _conts_max:
                        contributions.append(
                            cont
                        )
                    else:
                        contributions.append(
                            _conts[_new_cont_idx]
                        )

        _test.append(contributions.copy())

    contribution_df = pd.DataFrame(_test).T
    contribution_df.columns = hc.columns

    return contribution_df.round()


In [ ]:
contribution_df = calc_contribution(hc, ci, test)
contribution_df

In [ ]:
cnt = 0

sum_df = df.sum().round()
cont_mean_df = contribution_df.mean().round()
_UFO = list()
_UFO_cont = list()

for col in df:
    _me = sum_df[col]
    targets = sum_df[sum_df > _me].index
    
    for t_col in targets:
        _target = sum_df[t_col]
        me_cont = cont_mean_df[col]
        t_cont = cont_mean_df[t_col]
        
        if me_cont > t_cont:
            cnt += 1
            if t_col not in _UFO:
                _UFO.append(t_col)
                _UFO_cont.append(t_cont)
            print(col, t_col,_me, _target ,me_cont, t_cont, "오차 : {}".format(me_cont - t_cont))
            

print("에러 개수 : {}".format(cnt))
print("미확인 가구({}) : {}".format(len(_UFO),_UFO))
print("{}".format(_UFO_cont))

for _ufo_idx, value in enumerate(_UFO):
    UFO_NAME = _UFO[_ufo_idx]
    UFO_cont = _UFO_cont[_ufo_idx]

    sum_df = df.sum()
    
    
    UFO_kwh = sum_df[UFO_NAME]
    
    now_idx = [_ not in _UFO for _ in df.columns]
    now_cont = sum_df[now_idx][cont_mean_df == UFO_cont]
    next_cont = sum_df[cont_mean_df == (UFO_cont + 1)]
    
    print("--------------------------------")
    print("현재 cont rank: {}".format(UFO_cont))
    print("현재 cont max: {}kWh".format(now_cont.max()))
    print("UFO : {}kWh".format(UFO_kwh))
    print("다음 cont rank : {}".format(UFO_cont + 1))
    print("다음 cont min : {}kWh".format(next_cont.min()))
    print()
    
    now_err = abs(now_cont.max() - UFO_kwh)
    next_err = abs(next_cont.min() - UFO_kwh)
    print("해당 가구({})는 {} rank {} 가 적합합니다.".format(
        UFO_NAME,
      "다음" if  now_err > next_err else "현재",
       UFO_cont + 1 if now_err > next_err else UFO_cont
    ))
    
    if now_err > next_err:
        cont_mean_df[UFO_NAME] += 1
    
    print("--------------------------------\n")

In [ ]:
# 실제(합산치 상에서의) 최소, 최대 사용량 가구
cols = df.columns

sum_df = (df.sum() * 1000).round() / 1000
min_h = cols[sum_df.argmin()]
max_h = cols[sum_df.argmax()]
print("실제 (합산치 상에서의) 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) / 최대: {} ({}kwh)\n".format(
    min_h,
    sum_df[min_h],
    max_h,
    sum_df[max_h]
))

mean_cont_df = contribution_df.mean().round().astype("int")
_min_h = cols[mean_cont_df.argmin()]
_max_h = cols[mean_cont_df.argmax()]
print("기여도 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) (기여도: {}%) / 최대: {} ({}kwh) (기여도: {}%)\n".format(
    _min_h,
    sum_df[min_h],
    mean_cont_df[_min_h],
    _max_h,
    sum_df[max_h],
    mean_cont_df[_max_h]
))
print("- 최소사용량 가구 기여도 , 최대사용량 가구 기여도 비교 {}배".format(round(mean_cont_df[_max_h] / mean_cont_df[_min_h])))